In [ ]:
import os
os.environ["QT_QPA_PLATFORM"] = "offscreen"
from qgis.core import QgsApplication
app = QgsApplication([], False)
app.setPrefixPath("/apps/anvil/external/apps/qgis/3.40.1-Bratislava", True)
# Use the actual plugin path
plugin_path = os.path.expanduser("~/.local/share/QGIS/QGIS3/profiles/default/python/plugins")
app.setPluginPath(plugin_path)
app.initQgis()

In [ ]:
import os
import sys
if plugin_path not in sys.path:
	sys.path.append(plugin_path)

import processing
from processing.core.Processing import Processing

try:
    from processing_umep.processing_umep_provider import ProcessingUMEPProvider
    umep_provider = ProcessingUMEPProvider()
    QgsApplication.processingRegistry().addProvider(umep_provider)
    print("UMEP imported")
except Exception as e:
	print("UMEP import error:", e)

In [ ]:
import os
import time
from multiprocessing import Pool, cpu_count
from osgeo import gdal
import processing  # make sure your QGIS Python path is set so this imports
gdal.SetConfigOption('GDAL_DISABLE_STATS', 'YES')

#––– PARAMETERS –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
INPUT_FILE = "/anvil/projects/x-cis250634/test/atlanta_DSM_reprojected.tif"
OUTPUT_BASE = "/anvil/projects/x-cis250632/test/Wall_Aspect_Height/1500_WAH_multi"
OUTPUT_HEIGHT_PATH = ""
OUTPUT_ASPECT_PATH = ""
TILE_SIZE  = 1500
BUFFER     = 200
N_PROCS    = 40  
#–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

os.makedirs(OUTPUT_BASE, exist_ok=True)
start = time.time()

In [ ]:
def process_tile(tile_coords):
    x0, y0 = tile_coords
    ds = gdal.Open(INPUT_FILE)
    cols = ds.RasterXSize
    rows = ds.RasterYSize

    # compute window with buffer
    x_start = max(x0 - BUFFER, 0)
    y_start = max(y0 - BUFFER, 0)
    x_end   = min(x0 + TILE_SIZE + BUFFER, cols)
    y_end   = min(y0 + TILE_SIZE + BUFFER, rows)
    width   = x_end - x_start
    height  = y_end - y_start

    tile_name = f"tile_x{x0}_y{y0}"
    temp_tile = os.path.join(OUTPUT_BASE, f"{tile_name}.tif")
    height_out = os.path.join(OUTPUT_BASE, f"{tile_name}_wall_height.tif")
    aspect_out = os.path.join(OUTPUT_BASE, f"{tile_name}_wall_aspect.tif")

    try:
        # 1) extract buffered tile
        gdal.Translate(temp_tile, ds,
                       srcWin=[x_start, y_start, width, height],
                       format="GTiff",
                       creationOptions=["COMPRESS=LZW"])

        # 2) run UMEP wall‐height/aspect
        start = time.time()
        processing.run("umep:Urban Geometry: Wall Height and Aspect", {
            'INPUT': temp_tile,
            'INPUT_LIMIT': 3,
            'OUTPUT_HEIGHT': height_out,
            'OUTPUT_ASPECT': aspect_out
        })

        # 3) crop out buffer
        crop_x = x0 - x_start
        crop_y = y0 - y_start
        crop_w = min(TILE_SIZE, cols - x0)
        crop_h = min(TILE_SIZE, rows - y0)

        for out_path in (height_out, aspect_out):
            cropped = out_path.replace(".tif", "_cropped.tif")
            gdal.Translate(cropped, out_path,
                           srcWin=[crop_x, crop_y, crop_w, crop_h],
                           format="GTiff",
                           creationOptions=["COMPRESS=LZW"])
            os.replace(cropped, out_path)

        elapsed = round(time.time() - start, 2)
        print(f"{tile_name} done in {elapsed} sec")

    except Exception as e:
        print(f"Failed on {tile_name}: {e}")

    finally:
        ds = None  # close dataset
        
def merge_tiles(input_folder, output_path, component):
    # Get list of all cropped wall height or aspect tiles
    tif_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder)
                 if f.endswith(f"{component}.tif")]

    if not tif_files:
        raise ValueError(f"No matching .tif files found in {input_folder}")

    vrt_path = output_path.replace(".tif", ".vrt")

    # Build VRT (virtual mosaic)
    gdal.BuildVRT(vrt_path, tif_files)

    # Translate VRT to single output TIFF
    gdal.Translate(output_path, vrt_path, creationOptions=["COMPRESS=LZW"])

    print(f"Saved stitched raster to {output_path}")


In [ ]:
if __name__ == "__main__":
    # build list of top‐left coordinates for each tile
    ds = gdal.Open(INPUT_FILE)
    cols, rows = ds.RasterXSize, ds.RasterYSize
    ds = None

    tile_coords = [
        (x0, y0)
        for x0 in range(0, cols, TILE_SIZE)
        for y0 in range(0, rows, TILE_SIZE)
    ]

    print(f"Dispatching {len(tile_coords)} tiles on {N_PROCS} processes…")
    with Pool(processes=N_PROCS) as pool:
        pool.map(process_tile, tile_coords)

    print(f"All tiles submitted.: {time.time() - start}.s ")

    # Output files
    height_t = time.time()
    merge_tiles(OUTPUT_BASE, OUTPUT_HEIGHT_PATH, 'height')
    print(f"All height files merged: {time.time() - height_t}.s ")

    asepct_t = time.time()
    merge_tiles(OUTPUT_BASE, OUTPUT_ASPECT_PATH, 'aspect')
    print(f"All aspect files merged: {time.time() - asepct_t}.s ")